# Плоский канал. Часть 2

Я поправил функцию rp. До этого на оси трубы были точки, в которых значение завихренности не вычислялось, но потом использовалось. В геометрии трубы значения в этих точках входили умноженными на радиус и не имели значения. На сколько я помню, проблема была в этом. 

Сейчас можно рассчитывать, что программа работает верно. 

## Аргументы в пользу того, что программа работает верно

Сейчас у меня три версии кода. Первый - исходный код для трубы. Второй - код с параметром, плавно переводящим геометрию трубы в геометрию плоского канала. Третий - для плоского канала. В нем удалены переменные yt, rt, чтобы гарантировать, что они равны 1. При этом многие выражения упрощаются, соответственно, их проще проверить. 

1. В круглой трубе результаты расчета первой и второй программы совпадают. Аналогично, в плоском канале совпадают результаты второй и третьей программы.
2. В функции rp больше нет точек, которые участвуют в расчете не инициализированными. Чтобы в этом убедиться, я каждый массив перед ее вычислением заполнил значениями 'nan'. Если хотя бы в одной точке, которая участвует в расчете, это значение не будет перезаписано нормальным числом, то 'nan' попадет в расчет и заполнит все. Во внутренних точках значения вычисляются тем же кодом, что и раньше, так что ошибки тут быть не должно. На границе же точки инициализируются достаточно очевидным способом, и это тоже легко проверить.
3. Функция pres работает, после ее применения во всех случаях div < 1e-14 примерно
5. Функция lin повышает допустимый шаг по времени. Кроме этого, я исправил ошибочку, как мне кажется, и в исходной версии кода, где на оси трубы для u и w стояло условие, что они должны быть равны 0, которое я заменил на условие равенства нулю их производных по радиусу. Теперь, если считать с маленьким шагом по времени с функцией lin и без нее, получаемые поля скорости отличаются на порядок меньше, чем до этого. 
6. Весь другой код я тоже посматривал на предмет ошибок. 

## Перевод решения из трубы в канал

Поправив программу, я снова попробовал перевести решение из трубы в плоский канал методом Ньютона. И на этот раз удачно! Как и в прошлый раз, переводил новую бегущую волну, полученную из турбулентного течения (Re=2200, Xmax=5R, nsym=2). Меняя кривизну трубы, я фиксировал скорость сноса (cf=0.65U), так как хотел сохранить расстояние от стенки. На самоопределение же оставил ширину расчетной области Zmax. Заранее непонятно, какой она должна быть. 

На графике ниже приведено, как менялась ширина расчетной области Zmax при переходе от трубы к каналу. Для трубы Zmax = pi / nsym - ширина расчетной области при r = 1. По большому счету, приводить эту кривую нет смысла, так как я не уточнял, как именно меняется программа и расчетная область при 0 < c < 1. Значение имеет только ширина расчетной области при c = 1 в геометрии канала. 

<img src="transform_succ.png">

Ширина уменьшилась до 0.8 R. На графике ниже приведены средние вдоль трубы. 

<img src="newTW_means.png">

Решение больше не похоже на то, с которого я начинал в трубе ([newTW-Re2200](../nk/tw2200new/newTW-Re2200.ipynb)). Если раньше полосы боли ярко выражены (максимум продольной скорости достигался на полосе ускорения, а не на оси трубы), то теперь это не так. Продольная завихренность занимает очень небольшую область. Тем не менее, амплитуда пульсаций и продольных вихрей даже немного увеличилась (максимум продольной завихренности в 3.5 раза больше, но он достигается на стенке трубы. Амплитуда основных вихрей практически не изменилась). Кроме все прочего, основное свойство этой бегущей волны, что турбулентное течение мало от нее отходит, потеряно. В расчетах течение быстро ламинаризируется. Тем не менее, за формирование продольной завихренности отвечает все тот же механизм. 

На рисунке ниже приведен вклад в энергию OX. Все величины умножены на OX - среднюю продольную завихренность. В первой строке вклад от пульсаций, во второй - от среднего течения. 

<img src="newTW_ox_gen.png">

У этого решения есть и другие недостатки. Среднее течение оказывается неустойчивым, но наиболее быстро растущим оказывается возмущение с длиной волны Xmax/2. Наверное возмущение с длиной волны Xmax тоже не устойчиво, но я не проверял. Метод Ньютона позволяет найти такие решения, где главная гармоника (в данном случае двойной частоты) подавлена, но это отдаляет решение от действительности. В общем, это решение меня не очень удовлетворило, и переносить решения из трубы в канал таким методом наверное больше не стоит. 

## Движение по числу Рейнольдса

Чтобы найти более простое и, возможно, более устойчивое решение, я попробовал уменьшить число Рейнольдса, увеличивая при этом ширину расчетной области. На графике ниже приведено, как менялась Zmax от Re. Первое решение находится при Re=2200 примерно, красная черточка - последнее найденное решение с Zmax=1. 

<img src="newTW-Re_change.png">

Решение существует в очень широком диапазоне чисел Рейнольсда, Re=800 - это не придел. На графике ниже средние величины при Re=800, Zmax=1, примерно.

<img src="newTW-Re800-means.png">

Решение действительно стало проще. Полосы повышенной и пониженной скорости теперь совсем не выражены, зато пульсации уходят от стенки, и остаются только между полосами.

## Решение на сепаратрисе

Решение на сепаратрисе я тоже пересчитал, но ничего по сути не изменилось. 

<img src="edge_means.png">

Пульсации снова на верхней стенке. Может быть, можно искать какую-нибудь другую сепаратрису, например фиксировать не просто факт наличия пульсаций, а факт наличия пульсаций около нижней стенки. Тогда эта сепаратриса бы уже не удовлетворяла условию. Из этого бы даже статья могла бы получиться: обычная сепаратриса дает решения, не очень похожие на турбулентность, но вот способ найти сепаратрису по новому, которые дает пристенные решения. 

Здесь снова изолинии скорости подходят под углом к верхней стенки. Я подумал, что это может быть нормально. Если $\partial u/ \partial r = 0$ на верхней стенке, то около нее $\partial u/ \partial r$ мало. Но и $\partial u/ \partial z$ может быть мало и сравнимо с $\partial u/ \partial r$. Тогда grad u может иметь любое направление, и соответственно изолинии могут подходить под любым углом. 

## Заключение

Мне кажется, перенесенная в плоский канал бегущая волна оказывается очень искусственной в новой геометрии. Можно представить, что были особенности, нормальные для трубы, но в плоском канале бы они никогда не возникли, а метод Ньютона их сюда перенес все равно. Можно попробовать найти другие решение, вытащив их из турбулентного течения, например, как это было сделано в трубе, или предложенным вами методом. Но, по существу, это все не важно. Важно, что во всех случаях принцип формирования продольной завихренности оказывается один и тот же, и его можно исследовать на любом из решений. 

Я узнал о существовании метода обратной итерации для нахождения периодических решений. Он может быть эффективнее, чем метод Ньютона-Крылова, но я пока не разбирался с этим.